# Analyze Initial String

With this notebook you will be able to see if the initial string obtained from steering simulations (or the configurations you got some other way) are following the `string0.txt`. You will also be able to visualize the structures of the beads and compare them to the starting and final configurations.

In [ ]:
import glob
import re
import MDAnalysis as mda
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os 
from math import ceil
import nglview as nv
import shutil


def natural_sort(l):
    """
    Takes as input a list l of strings and sorts it with natural order.
      Parameters
      ----------
      l: list of strings.
      Returns
      -------
      l sorted
    """
    assert isinstance(l, list), "l is not a list!"
    for i in l:
        assert isinstance(i, str), "List contains non-string elements."
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(l, key=alphanum_key)

In [ ]:
def distance_atom_groups(u, sel1, sel2, progressbar=True, center_of_mass=False):
    """
    Calculate the distance between the centers of geometry (or mass) between two groups (sel1, sel2) as a function of time in the trajectory trj.

    Parameters
    ----------
    u: MDA universe to analyz trajectory to analyze.
    sel1: MDA selection containing at least 1 atom.
    sel2: MDA selection containing at least 1 atom.
    center_of_mass: Use the center of mass instead of center of geometry.
    progressbar: Show progressbar.

    Returns
    -------
    d: matplotlib figure object.
    """
    from MDAnalysis import Universe
    from MDAnalysis import AtomGroup
    from numpy import array
    from tqdm import tqdm
    from numpy.linalg import norm

    assert isinstance(u, Universe), "u should be a MDAnlaysis universe."
    assert isinstance(sel1, AtomGroup), "sel1 should be a MDAnlaysis universe."
    assert isinstance(sel2, AtomGroup), "sel2 should be a MDAnlaysis universe."
    assert isinstance(progressbar, bool), "progressbar should be boolean."
    assert sel1.n_atoms >= 1, "sel1 should have at least 1 atom."
    assert sel2.n_atoms >= 1, "sel2 should have at least 1 atom."

    d = []
    for i, ts in tqdm(
        enumerate(u.trajectory), total=u.trajectory.n_frames, disable=not progressbar
    ):
        if center_of_mass:
            csel1 = sel1.center_of_mass()
            csel2 = sel2.center_of_mass()
        else:
            csel1 = sel1.centroid()
            csel2 = sel2.centroid()
        d.append([ts.dt * i, norm(csel1 - csel2)])
    return array(d)

Choose working directory

In the cell bellow you can select which will be the simulation directory (in case this notebook is elsewhere). If the notebook is in the simulation directory just leave it as ".".

In [ ]:
%ls ../data/raw

In [ ]:
simulation_directory = "/data/sperez/Projects/string_sims/data/raw/C2I_v1_amber/"
os.chdir(simulation_directory)
os.getcwd()

Read in the bead paths and cv variables.

In [ ]:
beads = ["0","2","4","6","8","10","12","14","16","18","20","22","24","26","28" ,"30","32", "33"]
for i in range(len(beads)):
    beads[i] = f"0_save/{beads[i]}/restrained"

In [ ]:
beads = natural_sort(glob.glob("md/1/*/restrained"))

In [ ]:
beads = natural_sort(glob.glob("0_34beads/*/restrained"))

In [ ]:
cvs, ndx_groups = pickle.load(open("cv_steer.pkl", "rb"))

In [ ]:
cvs, ndx_groups = pickle.load(open("cv.pkl", "rb"))

In [ ]:
len(ndx_groups)

Calculate the values of the CVs in the `md/0/*/restrained/confout.gro` files.

In [ ]:
topology = 'topology/5VKH.pdb'
dis = []

for b in beads[:]:

    if topology != None:
        u = mda.Universe(topology, b + "/confout.gro")
    else:
        u = mda.Universe(b + "/confout.gro")
    d = []
    for cv in cvs:
        d.append(
            distance_atom_groups(
                u,
                u.select_atoms(ndx_groups[list(ndx_groups.keys())[cv[0] - 1]]),
                u.select_atoms(ndx_groups[list(ndx_groups.keys())[cv[1] - 1]]),
                progressbar=False,
            )[-1, 1]
        )
    dis.append(np.array(d))
dis = np.array(dis) / 10

In [ ]:
#shutil.copy('strings/string0.txt','strings/string_steer.txt')
#np.savetxt('strings/string0.txt', dis)

## Plot of `string0.txt` versus the configs that will be used.

The value of the cvs in the `confout.gro` should approximatelly follow `string0.txt`. If it doesn't you can increase the steering `kappa` and also make the steering simulation longer by increasing the `nsteps` per bead.

In [ ]:
string = np.loadtxt("strings/string_steer.txt")
string = np.loadtxt("strings/string0.txt")
n_plots = string.shape[1]
fig, ax = plt.subplots(ceil(n_plots / 2), 2, figsize=(15, 4 * ceil(n_plots / 2)))
ax = ax.flatten()
for i in range(n_plots):
    ax[i].plot(string[:, i], ls="", marker="x", label="string0")
    ax[i].set_xlabel(
        f"{list(ndx_groups.keys())[2*i]} - {list(ndx_groups.keys())[2*i+1]}", size=16
    )
    ax[i].set_ylabel("d (nm)", size=16)
    ax[i].tick_params(axis="both", which="major", labelsize=13)
bead_numbers = [int(b.split("/")[2]) for b in beads]
if len(dis) != 0:
    for i in range(n_plots):
        ax[i].plot(bead_numbers, dis[:, i], ls="", marker=".", label="confout")
        ax[i].set_title(f"cv{i}")
ax[1].legend()
if n_plots % 2:
    fig.delaxes(ax[-1])
fig.tight_layout()